In [ ]:
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
import time
import pickle
import shutil
import random
from  urllib.request import urlopen

In [ ]:
# Paths
DATA_PATH = '../data/'
OUTPUT_PATH = '../data/raw/'
CREDS_PATH = '../collection/credentials/'

In [ ]:
# Check files in data folder
datafiles = [f for f in listdir(OUTPUT_PATH) if isfile(join(OUTPUT_PATH, f))]
print(datafiles)

In [ ]:
# Load df
load_file = datafiles[1]

df = pd.read_json(OUTPUT_PATH + load_file, convert_dates=True, lines=True, orient='records')

In [ ]:
df.head(1)

In [ ]:
# Checking the data we need
df.links[1]

In [ ]:
# Creating name for directory creation

QUERY = load_file.split('_')[1].split('.')[0]

In [ ]:
# Make folder to hold PDFS
pdf_dir = DATA_PATH + QUERY + 'pdfs'

if not os.path.exists(pdf_dir): 
  os.makedirs(pdf_dir)


In [ ]:
# Time out for our requests
timeout_secs = 10 

# List of PDFs we have
have = set(os.listdir(pdf_dir))

# Current and total downloads
numok = 0
numtot = 0

for index, row in df.iterrows():
  for item in row.links:
    if item['type'] == 'application/pdf':
      pdf_url = item['href']
      basename = pdf_url.split('/')[-1]
      fname = os.path.join(pdf_dir, basename)
      print(fname)

      try:
          if not basename in have:
            print('fetching %s into %s' % (pdf_url, fname))
            req = urlopen(pdf_url, None, timeout_secs)
            with open(fname, 'wb') as fp:
                shutil.copyfileobj(req, fp)
            time.sleep(0.05 + random.uniform(0,0.1))
          else:
            print('%s exists, skipping' % (fname, ))
          numok+=1
      except Exception as e:
          print('error downloading: ', pdf_url)
          print(e)
        
      print('%d/%d of %d downloaded ok.' % (numok, numtot, len(df)))
        
print('final number of papers downloaded okay: %d/%d' % (numok, len(df)))